In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.chained_assignment", None)

In [2]:
#reading all the datasets into a pandas dataframe
events = pd.read_json("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Figshare/events_England.json", encoding = "unicode_escape")
matches = pd.read_json("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Figshare/matches_England.json", encoding = "unicode_escape")
teams = pd.read_json("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Figshare/teams.json", encoding = "unicode_escape")
players = pd.read_json("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Figshare/players.json", encoding = "unicode_escape")
event_id_tagger = pd.read_csv("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Figshare/eventid2name.csv", encoding = "unicode_escape")
event_tag2name = pd.read_csv("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Figshare/tags2name.csv", encoding = "unicode_escape")

In [ ]:
events.head()

In [26]:
event_id_tagger

,event,subevent,event_label,subevent_label
0,1,10,Duel,Air duel
1,1,11,Duel,Ground attacking duel
2,1,12,Duel,Ground defending duel
3,1,13,Duel,Ground loose ball duel
4,2,20,Foul,Foul
5,2,21,Foul,Hand foul
6,2,22,Foul,Late card foul
7,2,23,Foul,Out of game foul
8,2,24,Foul,Protest
9,2,25,Foul,Simulation


In [ ]:
matches.head(1)

In [ ]:
teams.head()

In [28]:
players.head(1)

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687


In [27]:
event_tag2name

,Tag,Label,Description
0,101,Goal,Goal
1,102,own_goal,Own goal
2,301,assist,Assist
3,302,keyPass,Key pass
4,1901,counter_attack,Counter attack
5,401,Left,Left foot
6,402,Right,Right foot
7,403,head/body,Head/body
8,1101,direct,Direct
9,1102,indirect,Indirect


## Building Pass network edge dataframe for last 5 matches

In [3]:
#finding last 5 match id's for all teams
matches_last5 = matches[matches['gameweek'].isin([33, 34, 35, 36, 37])]
matches_last5 = matches_last5.sort_values(["gameweek", "date"])
matches_last5_list = matches_last5['wyId'].values.tolist()

## Extracting tags from tags coloumn

In [4]:
#Functino to split and append only tag id's to a list
def tags_splitter(row):
    tags_list = []
    for col in row.keys():
        if row[col]:
            tags_list.append(row[col]['id'])
    return tags_list


#splitting all tags fields into a seperate coloumn in list format
df = events[['tags']].reset_index()
df['split_tags'] = pd.DataFrame(events.tags.values.tolist()).add_prefix('tag_').apply(tags_splitter, axis=1)

#merging splited tag list column to events dataframe
events = events.reset_index()
events_tagged = events.merge(df, on='index', how='inner').drop(['index', 'tags_y'], axis=1)
events_tagged

,eventId,subEventName,tags_x,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,split_tags
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,[1801]
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,[1801]
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,[1801]
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,[1801]
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,[1801]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
643145,5,Ball out of the field,[],0,"[{'y': 32, 'x': 0}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2796.732525,50,251596409,[]
643146,3,Corner,"[{'id': 302}, {'id': 801}, {'id': 1801}]",70965,"[{'y': 100, 'x': 100}, {'y': 47, 'x': 88}]",2500098,Free Kick,1633,2H,2829.821084,30,251596232,"[302, 801, 1801]"
643147,1,Air duel,"[{'id': 701}, {'id': 1802}]",7919,"[{'y': 53, 'x': 12}, {'y': 50, 'x': 14}]",2500098,Duel,1623,2H,2831.211419,10,251596410,"[701, 1802]"
643148,1,Air duel,"[{'id': 703}, {'id': 1801}]",8005,"[{'y': 47, 'x': 88}, {'y': 50, 'x': 86}]",2500098,Duel,1633,2H,2832.434399,10,251596234,"[703, 1801]"


## Last 5 matches

In [9]:
#filtering for last 5 matches
events_tagged_last5 = events_tagged[events_tagged['matchId'].isin(matches_last5_list)][['id', 'matchId', 'teamId', 'playerId', 'eventId', 'eventName', 
                                                                                        'subEventId', 'subEventName','positions', 'matchPeriod','eventSec', 'split_tags']]
events_tagged_last5

,id,matchId,teamId,playerId,eventId,eventName,subEventId,subEventName,positions,matchPeriod,eventSec,split_tags
541765,240736735,2500039,1659,7989,8,Pass,85,Simple pass,"[{'y': 43, 'x': 42}, {'y': 96, 'x': 38}]",1H,4.687259,[1801]
541766,240736737,2500039,1659,9279,8,Pass,85,Simple pass,"[{'y': 96, 'x': 38}, {'y': 77, 'x': 21}]",1H,7.569245,[1801]
541767,240736738,2500039,1659,9277,8,Pass,85,Simple pass,"[{'y': 77, 'x': 21}, {'y': 39, 'x': 25}]",1H,9.516850,[1801]
541768,240736739,2500039,1659,134102,8,Pass,85,Simple pass,"[{'y': 39, 'x': 25}, {'y': 16, 'x': 34}]",1H,11.715943,[1801]
541769,240736741,2500039,1659,9285,8,Pass,85,Simple pass,"[{'y': 16, 'x': 34}, {'y': 41, 'x': 26}]",1H,14.241886,[1801]
...,...,...,...,...,...,...,...,...,...,...,...,...
626719,249307782,2500088,1624,8717,10,Shot,100,Shot,"[{'y': 59, 'x': 90}, {'y': 100, 'x': 100}]",2H,3021.640087,"[403, 2101, 201, 1802]"
626720,249307798,2500088,1627,5281,7,Others on the ball,71,Clearance,"[{'y': 44, 'x': 9}, {'y': 39, 'x': 3}]",2H,3022.979218,"[1401, 1802]"
626721,249307774,2500088,1624,61967,10,Shot,100,Shot,"[{'y': 61, 'x': 97}, {'y': 100, 'x': 100}]",2H,3024.562347,"[402, 201, 1210, 1802]"
626722,249307577,2500088,1627,0,5,Interruption,50,Ball out of the field,"[{'y': 42, 'x': 0}, {'y': 0, 'x': 0}]",2H,3057.453741,[]


## 90-min time conversion

In [10]:
events_tagged_last5['eventSec'] = events_tagged_last5.apply(lambda x: (45 + int(np.ceil(x['eventSec']/60))) if x['matchPeriod'] == '2H' else int(np.ceil(x['eventSec']/60)), axis=1)
events_tagged_last5

,id,matchId,teamId,playerId,eventId,eventName,subEventId,subEventName,positions,matchPeriod,eventSec,split_tags
541765,240736735,2500039,1659,7989,8,Pass,85,Simple pass,"[{'y': 43, 'x': 42}, {'y': 96, 'x': 38}]",1H,1,[1801]
541766,240736737,2500039,1659,9279,8,Pass,85,Simple pass,"[{'y': 96, 'x': 38}, {'y': 77, 'x': 21}]",1H,1,[1801]
541767,240736738,2500039,1659,9277,8,Pass,85,Simple pass,"[{'y': 77, 'x': 21}, {'y': 39, 'x': 25}]",1H,1,[1801]
541768,240736739,2500039,1659,134102,8,Pass,85,Simple pass,"[{'y': 39, 'x': 25}, {'y': 16, 'x': 34}]",1H,1,[1801]
541769,240736741,2500039,1659,9285,8,Pass,85,Simple pass,"[{'y': 16, 'x': 34}, {'y': 41, 'x': 26}]",1H,1,[1801]
...,...,...,...,...,...,...,...,...,...,...,...,...
626719,249307782,2500088,1624,8717,10,Shot,100,Shot,"[{'y': 59, 'x': 90}, {'y': 100, 'x': 100}]",2H,96,"[403, 2101, 201, 1802]"
626720,249307798,2500088,1627,5281,7,Others on the ball,71,Clearance,"[{'y': 44, 'x': 9}, {'y': 39, 'x': 3}]",2H,96,"[1401, 1802]"
626721,249307774,2500088,1624,61967,10,Shot,100,Shot,"[{'y': 61, 'x': 97}, {'y': 100, 'x': 100}]",2H,96,"[402, 201, 1210, 1802]"
626722,249307577,2500088,1627,0,5,Interruption,50,Ball out of the field,"[{'y': 42, 'x': 0}, {'y': 0, 'x': 0}]",2H,96,[]


## finding the from and to passer

In [ ]:
sender = events_tagged_last5.iloc[:-1].reset_index(drop=True)
receiver = events_tagged_last5.iloc[1:][['matchId','teamId','playerId']].reset_index(drop=True).rename(columns={'matchId':'matchId_receiver', 
                                                                                                                'teamId':'teamId_receiver', 
                                                                                                                'playerId':'playerId_receiver'})

In [ ]:
events_combined = pd.concat([sender, receiver], axis=1)
pass_combined = events_combined[events_combined['eventId']==8][['matchId', 'teamId', 'playerId',
       'subEventId', 'subEventName', 'positions', 'matchPeriod',
       'eventSec', 'split_tags', 'matchId_receiver', 'teamId_receiver','playerId_receiver']]

In [ ]:
#removing rows where matchId are not equal
pass_combined = pass_combined.loc[(pass_combined['matchId'] == pass_combined['matchId_receiver'])]

#removing rows where teamId are not equal
pass_combined = pass_combined.loc[(pass_combined['teamId'] == pass_combined['teamId_receiver'])][['matchId', 'teamId', 'playerId', 'playerId_receiver', 'subEventId', 'subEventName',
       'positions', 'matchPeriod', 'eventSec', 'split_tags']]

In [ ]:
pass_combined

## Adding substitutes data to the events data

## Go to the bottom of the notebook and run the cells to substitute data before running this part

In [ ]:
#adding the changed sub passes to the actual passes
pass_combined = pd.concat([pass_combined, subs_pass_combined])

pass_combined

In [ ]:
#reading full players list from saved final_player_list.csv created from further below code snippets
#players_last5 = pd.read_csv("final_player_list.csv").drop('Unnamed: 0', axis=1)

In [ ]:
players_last5 = pd.read_excel("final_player_list_final_excel.xlsx", sheet_name='final_player_list').drop('Unnamed: 0', axis=1)
players_last5.head()

In [ ]:
#keeping entries for : passed by players in the starting 11
temp_pass1 = pass_combined.merge(players_last5)[['matchId', 'teamId', 'playerId', 'shortName', 'position', 'X', 'Y','playerId_receiver', 'subEventId',
       'subEventName', 'positions', 'matchPeriod', 'eventSec',
       'split_tags']].rename(columns={'shortName':'shortName_sender', 'position':'position_sender',
                                     'X':'X_sender', 'Y':'Y_sender'})

#keeping entries for : passes received by players in the starting 11
pass_network_comp = temp_pass1.merge(players_last5, left_on=['matchId', 'teamId', 'playerId_receiver'], right_on=['matchId', 'teamId', 'playerId']).rename(columns={'playerId_x':'playerId_sender', 'shortName':'shortName_receiver', 
'position':'position_receiver', 'X':'X_receiver', 'Y':'Y_receiver'})[['matchId', 'teamId', 'playerId_sender', 'shortName_sender', 'position_sender', 'X_sender', 'Y_sender', 'playerId_receiver', 'shortName_receiver', 
  'position_receiver', 'X_receiver', 'Y_receiver', 'subEventName', 'matchPeriod', 'eventSec', 'split_tags']].sort_values(['matchId','teamId'])

#exploding the tags field to seperate lines
pass_network_tag = pass_network_comp.explode('split_tags').rename(columns={'split_tags':'Tag'})

#obtaining the tags names
pass_network = pass_network_tag.merge(event_tag2name[['Tag','Label']], how="left").drop('Tag', axis=1)

pass_network

In [ ]:
def time_cat(x):
    if x >= 0 and x <= 15:
        return '0-15'
    elif x > 15 and x <= 30:
        return '16-30'
    elif x > 30 and x <= 45:
        return '31-45'
    elif x > 45 and x <= 60:
        return '46-60'
    elif x > 60 and x <= 75:
        return '61-75'
    else:
        return '76-90+'
pass_network['timeCat'] = pass_network['eventSec'].apply(lambda x: time_cat(x))

pass_network

In [ ]:
#pass_network.to_csv("pass_network.csv")

In [ ]:
# final_pass_network = pass_network.groupby(['matchId','teamId','playerId_sender','shortName_sender','position_sender', 'X_sender', 'Y_sender',
#                       'playerId_receiver','shortName_receiver','position_receiver', 'X_receiver', 'Y_receiver']).agg({'matchPeriod':'count'}).reset_index()

# final_pass_network

In [ ]:
# final_pass_network.groupby(['matchId', 'playerId_sender']).agg({'matchPeriod':'sum'})

In [ ]:
pass_network[['matchId', 'teamId', 'playerId_sender', 'shortName_sender',
       'position_sender', 'X_sender', 'Y_sender', 'playerId_receiver',
       'shortName_receiver', 'position_receiver', 'X_receiver',
       'Y_receiver', 'subEventName', 'matchPeriod', 'eventSec', 'Label',
       'timeCat']]

## splitting tag to each line

In [7]:
events_tagged_last5 = events_tagged_last5.explode('split_tags')
events_tagged_last5 = events_tagged_last5.rename(columns={'split_tags':'tags'})
events_tagged_last5

,id,matchId,teamId,playerId,eventId,eventName,subEventId,subEventName,positions,matchPeriod,eventSec,tags
541765,240736735,2500039,1659,7989,8,Pass,85,Simple pass,"[{'y': 43, 'x': 42}, {'y': 96, 'x': 38}]",1H,1,1801
541766,240736737,2500039,1659,9279,8,Pass,85,Simple pass,"[{'y': 96, 'x': 38}, {'y': 77, 'x': 21}]",1H,1,1801
541767,240736738,2500039,1659,9277,8,Pass,85,Simple pass,"[{'y': 77, 'x': 21}, {'y': 39, 'x': 25}]",1H,1,1801
541768,240736739,2500039,1659,134102,8,Pass,85,Simple pass,"[{'y': 39, 'x': 25}, {'y': 16, 'x': 34}]",1H,1,1801
541769,240736741,2500039,1659,9285,8,Pass,85,Simple pass,"[{'y': 16, 'x': 34}, {'y': 41, 'x': 26}]",1H,1,1801
...,...,...,...,...,...,...,...,...,...,...,...,...
626721,249307774,2500088,1624,61967,10,Shot,100,Shot,"[{'y': 61, 'x': 97}, {'y': 100, 'x': 100}]",2H,96,201
626721,249307774,2500088,1624,61967,10,Shot,100,Shot,"[{'y': 61, 'x': 97}, {'y': 100, 'x': 100}]",2H,96,1210
626721,249307774,2500088,1624,61967,10,Shot,100,Shot,"[{'y': 61, 'x': 97}, {'y': 100, 'x': 100}]",2H,96,1802
626722,249307577,2500088,1627,0,5,Interruption,50,Ball out of the field,"[{'y': 42, 'x': 0}, {'y': 0, 'x': 0}]",2H,96,NaN


## Pass Network Dataset

In [ ]:
#filtering only pass data
pass_df = events_tagged_last5[events_tagged_last5['eventId'].isin([8])][['id', 'matchId', 'teamId', 'playerId', 'subEventId', 'subEventName',
                                                                         'positions', 'matchPeriod','eventSec', 'tags']]

pass_df

## Team data

## Match related info integration

## Player details

## Node positions

In [ ]:
def home_formation(row):
    #getting the match id
    wyId = row['wyId']
    
    #variables to store home team list
    hTeamPlayers = []
    
    #obtaining the team ids
    teams = list(row['teamsData'].keys())
    
    #home and away team details
    home_team_data = row['teamsData'][teams[0]]
    
    #game details
    row['teamId'] = teams[0]
    row['goalsScored'] = home_team_data['score']
    
    #retriving home lineup
    home_team_list = home_team_data['formation']['lineup']
    for players in home_team_list:
        hTeamPlayers.append(players['playerId'])
    #assigning it to 11 variables
    row['Player1'] = hTeamPlayers[0]
    row['Player2'] = hTeamPlayers[1]
    row['Player3'] = hTeamPlayers[2]
    row['Player4'] = hTeamPlayers[3]
    row['Player5'] = hTeamPlayers[4]
    row['Player6'] = hTeamPlayers[5]
    row['Player7'] = hTeamPlayers[6]
    row['Player8'] = hTeamPlayers[7]
    row['Player9'] = hTeamPlayers[8]
    row['Player10'] = hTeamPlayers[9]
    row['Player11'] = hTeamPlayers[10]
    
    #returning the required details
    return row




def away_formation(row):
    #getting the match id
    wyId = row['wyId']
    
    #variables to store away team list
    aTeamPlayers = []
    
    #obtaining the team ids
    teams = list(row['teamsData'].keys())
    
    #away team details
    away_team_data = row['teamsData'][teams[1]]
    
    #game details
    row['teamId'] = teams[1]
    row['goalsScored'] = away_team_data['score']
    
    #retriving away lineup
    away_team_list = away_team_data['formation']['lineup']
    for players in away_team_list:
        aTeamPlayers.append(players['playerId'])
    #assigning it to 11 variables
    row['Player1'] = aTeamPlayers[0]
    row['Player2'] = aTeamPlayers[1]
    row['Player3'] = aTeamPlayers[2]
    row['Player4'] = aTeamPlayers[3]
    row['Player5'] = aTeamPlayers[4]
    row['Player6'] = aTeamPlayers[5]
    row['Player7'] = aTeamPlayers[6]
    row['Player8'] = aTeamPlayers[7]
    row['Player9'] = aTeamPlayers[8]
    row['Player10'] = aTeamPlayers[9]
    row['Player11'] = aTeamPlayers[10]

    
    #returning the required details
    return row

In [ ]:
#finding starting 11 for home and away teams
df_home = matches[['wyId', 'gameweek', 'teamsData', 'winner']].apply(home_formation, axis=1).drop(['teamsData'], axis=1)
df_away = matches[['wyId', 'gameweek', 'teamsData', 'winner']].apply(away_formation, axis=1).drop(['teamsData'], axis=1)

#combining both to a single df
node_positions = df_home.append(df_away).sort_values('wyId', ascending=False)
node_positions

In [ ]:
#choosing only the last 5 matches
node_positions_last5 = node_positions[node_positions['gameweek'].isin([33,34,35,36,37])]
node_positions_last5

In [ ]:
node_positions_last5_expand = node_positions_last5.melt(id_vars=["wyId","gameweek","winner","teamId","goalsScored"], value_name="playerId").drop("variable", axis=1)
node_positions_last5_expand

## Teams in England

In [8]:
def area(row):
    if row['name'] == 'England':
        return True
    else:
        return False
    
teams['area_bool'] = teams['area'].apply(area)
teams_england = teams[teams['area_bool']].drop(['area_bool'], axis=1)[['wyId', 'name']]

#droping England national team
teams_england = teams_england.drop(122)

#adding swansea city(Wales) to the team list
teams_england.loc[19] = [10531, "Swansea City"]

teams_england = teams_england.sort_values('name').rename(columns={'wyId':'teamId', 'name':'teamName'})

teams_england

,teamId,teamName
26,1659,AFC Bournemouth
84,1609,Arsenal
27,1651,Brighton & Hove Albion
39,1646,Burnley
78,1610,Chelsea
70,1628,Crystal Palace
65,1623,Everton
8,1673,Huddersfield Town
54,1631,Leicester City
77,1612,Liverpool


## getting player names and club to use Fifa 18 dataset to get their positions

In [ ]:
#To remove the repeated players from the node_position dataframe
def player_ids(x):
    players_comb = x[['Player1','Player2','Player3','Player4','Player5','Player6',
                'Player7','Player8','Player9','Player10','Player11']].values.tolist()
    players_flat = [item for sublist in players_comb for item in sublist]
    teamPlayers = list(dict.fromkeys(players_flat))
    return teamPlayers

In [ ]:
#to get the players list from each team
selected0 = pd.DataFrame(node_positions_last5[['teamId', 'Player1','Player2','Player3','Player4','Player5','Player6',
                'Player7','Player8','Player9','Player10','Player11']].groupby('teamId').apply(lambda x: player_ids(x))).reset_index().rename(columns={0:'wyId'}).explode('wyId')


#merging with players dataset to get the player names
selected1 = selected0.merge(players, how="left")[['wyId', 'firstName','lastName', 'teamId', 'shortName']]
#converting column to int64
selected1['teamId'] = selected1['teamId'].astype(int)


#merging with the england teams dataset to get the team details
selectedPlayers = selected1.merge(teams_england, how="left")

selectedPlayers

In [ ]:
#reading epl 17/18 dataset
fifa_18 = pd.read_excel("/home/kenny/spring2021/IAT814/Project/Soccer/Dataset/Fifa 18/EPL_1718_Players.xlsx")
fifa_18 = fifa_18.rename(columns={'club':'teamName'})

In [ ]:
#To create shortname in the epl datatset
def short_name(row):
    name = row.split()
    firstname = name[0][0]
    lastname = name[-1]
    shortname = firstname + '. ' + lastname
    return shortname

fifa_18['shortName'] = fifa_18['name'].apply(short_name)
fifa_18

In [ ]:
#finding the common players
all_players = selectedPlayers.merge(fifa_18, how='left',indicator = True)
all_players

In [ ]:
common_players = all_players[all_players['_merge']=='both']
common_players

In [ ]:
nan_players = all_players[all_players['_merge']=='left_only'][['wyId','firstName','lastName','teamId','shortName','teamName','position']]
nan_players.to_csv('left_players.csv')
nan_players

In [ ]:
#found out the position and filled them
nan_players_filled = pd.read_csv("left_players_filled.csv")
nan_players_filled = nan_players_filled.drop('Unnamed: 0', axis=1)
nan_players_filled

In [ ]:
player_positions = pd.concat([common_players, nan_players_filled]).drop(['_merge', 'name'], axis=1)
player_positions

In [ ]:
final_player_list = node_positions_last5_expand.merge(player_positions, left_on="playerId", right_on="wyId", how="left")[['wyId_x','gameweek','winner','teamId_x','teamName',
                                                                                                      'goalsScored','playerId','shortName','position']].rename(columns={"wyId_x":"matchId", "teamId_x":"teamId"})

final_player_list = final_player_list.sort_values(["matchId", "teamName", "position"])
final_player_list

In [ ]:
final_player_list.to_csv("final_player_list.csv")

## Substitution Inclusive method

In [ ]:
def home_formation(row):
    #getting the match id
    wyId = row['wyId']
    
    #variables to store home team list
    subPlayers = []
    
    #obtaining the team ids
    teams = list(row['teamsData'].keys())
    
    #home and away team details
    home_team_data = row['teamsData'][teams[0]]
    
    #game details
    row['teamId'] = teams[0]
    row['goalsScored'] = home_team_data['score']
    
    #getting the substituition details
    home_subs_list = home_team_data['formation']['substitutions']  
    
    #appending subs
    if home_subs_list != 'null':
        for players in home_subs_list:
            subPlayers.append([players['playerIn'], players['playerOut']])
            
    row['subPlayer'] = subPlayers

    #returning the player lists
    return row



def away_formation(row):
    #getting the match id
    wyId = row['wyId']
    
    #variables to store home team list
    away_subPlayers = []
    
    #obtaining the team ids
    teams = list(row['teamsData'].keys())
    
    #home and away team details
    away_team_data = row['teamsData'][teams[1]]
    
    #game details
    row['teamId'] = teams[1]
    row['goalsScored'] = away_team_data['score']
    
    #getting the substituition details
    away_subs_list = away_team_data['formation']['substitutions']  
    
    #appending subs
    if away_subs_list != 'null':
        for players in away_subs_list:
            away_subPlayers.append([players['playerIn'], players['playerOut']])
            
    row['subPlayer'] = away_subPlayers

    #returning the player lists
    return row




def column_splitter(row):
    if row['subPlayer'] != None:
        row['playerIn'] = row['subPlayer'][0]
        row['playerOut'] = row['subPlayer'][1]
    
    return row

In [ ]:
#finding starting 11 for home and away teams
df_home = matches[['wyId', 'gameweek', 'teamsData', 'winner']].apply(home_formation, axis=1).explode('subPlayer').dropna().apply(column_splitter, axis=1).drop(['teamsData', 'subPlayer'], axis=1)
df_away = matches[['wyId', 'gameweek', 'teamsData', 'winner']].apply(away_formation, axis=1).explode('subPlayer').dropna().apply(column_splitter, axis=1).drop(['teamsData', 'subPlayer'], axis=1)
#combining both to a single df
node_positions = df_home.append(df_away).sort_values(['wyId', 'teamId'], ascending=False)
node_positions

In [ ]:
#choosing only the last 5 matches
node_positions_last5 = node_positions[node_positions['gameweek'].isin([33,34,35,36,37])]
node_positions_last5['teamId'] = node_positions_last5['teamId'].astype('int')
node_positions_last5 = node_positions_last5.drop(['winner', 'goalsScored'], axis=1)
node_positions_last5

In [ ]:
# #To remove the repeated players from the node_position dataframe
# def player_ids(x):
#     players_comb = x['playerId'].values.tolist()
#     teamPlayers = list(dict.fromkeys(players_comb))
#     return teamPlayers


In [ ]:
# #to get the players list from each team
# selected0 = pd.DataFrame(node_positions_last5_expand.groupby('teamId').apply(player_ids)).reset_index().rename(columns={0:'wyId'}).explode('wyId')

# #merging with players dataset to get the player names
# selected1 = selected0.merge(players, how="left")[['wyId', 'firstName','lastName', 'teamId', 'shortName']]
# #converting column to int64
# selected1['teamId'] = selected1['teamId'].astype(int)


# #merging with the england teams dataset to get the team details
# selectedPlayers = selected1.merge(teams_england, how="left")

# selectedPlayers

In [ ]:
subs_pass_combined = pass_combined.merge(node_positions_last5, left_on=['matchId', 'teamId', 'playerId'], 
                    right_on=['wyId', 'teamId', 'playerIn']).drop(['playerId', 'playerIn'], axis=1).rename(columns={'playerOut':'playerId'})

subs_pass_combined = subs_pass_combined[['matchId','teamId','playerId','playerId_receiver','subEventId','subEventName',
                    'positions','matchPeriod','eventSec','split_tags']]

subs_pass_combined

## Tag Id splitter

In [ ]:
#Functino to split and append only tag id's to a list
def tags_splitter(row):
    tags_list = []
    for col in row.keys():
        if row[col]:
            tags_list.append(row[col]['id'])
    return tags_list
    

#splitting all tags fields into a seperate coloumn in list format
df = events[['tags']].reset_index()
df['split_tags'] = pd.DataFrame(events.tags.values.tolist()).add_prefix('tag_').apply(tags_splitter, axis=1)

#merging splited tag list column to events dataframe
events = events.reset_index()
events = events.merge(df, on='index', how='inner').drop(['index', 'tags_y'], axis=1)

In [ ]:
events.head()

In [ ]:
def accurate_pass(x):
    if 1801 in x:
        return True
    else:
        return False

events_pass = events[events['split_tags'].apply(accurate_pass)]

## Score Cards

In [ ]:
def score_split(row):
    #seperating the team names and scores
    label_data = row['label'].split(', ')
    #obtaining the team names
    team_names = label_data[0].split(' - ')
    home_team = team_names[0]
    away_team = team_names[1]
    #obtaining the scores
    score = label_data[1]
    
    #adding the values to the coloumns
    row['home_team'] = home_team
    row['away_team'] = away_team
    row['score'] = score
    
    #returning the row values
    return row
    

#geting the match team names and scores
matches_scores = matches[['wyId', 'gameweek', 'label', 'winner']].rename(columns={'wyId':'matchId'})
matches_scores = matches_scores.apply(lambda x: score_split(x), axis=1).drop('label', axis=1)

#last 5 gameweeks
matches_scores = matches_scores[matches_scores['gameweek'].isin([33,34,35,36,37])].sort_values(['gameweek', 'matchId'])
matches_scores.head()

In [ ]:
#flipping the matches_scores to get full 20 teams on the home team
def match_score_expander(row):
    #flipping the home and away team names
    away_team = row['home_team']
    home_team = row['away_team']
    row['home_team'] = home_team
    row['away_team'] = away_team
    
    #flipping the scores
    score_list = row['score'].split(' - ')
    row['score'] = score_list[1]+' - '+score_list[0]
    
    return row

#flipping the matches_scores to get full 20 teams on the home team
matches_scores_flip = matches_scores.apply(lambda x: match_score_expander(x), axis=1)
matches_scores_flip.head()

In [ ]:
#concatening to get the full list
matches_scores = pd.concat([matches_scores, matches_scores_flip]).sort_values(['gameweek', 'matchId']).rename(columns={'away_team':'oppostion_team'})
matches_scores.head()

In [ ]:
#getting the teamId
matches_scores_temp = matches_scores.merge(teams_england, left_on='oppostion_team', right_on='teamName', how='left').rename(columns={'teamId':'oppostion_teamId'}).drop('teamName', axis=1)
matches_scores_temp = matches_scores_temp.merge(teams_england, left_on='home_team', right_on='teamName', how='left').drop('teamName', axis=1)[['matchId', 'gameweek', 'home_team', 'teamId',
                                                                                                                                              'oppostion_team', 'oppostion_teamId', 'score', 'winner']]
matches_scores_temp

In [ ]:
#finding win, loos or draw
def result(row):
    home_team = row['teamId']
    if row['winner'] == 0:
        row['result'] = 'Draw'
    elif home_team == row['winner']:
        row['result'] = 'Win'
    else:
        row['result'] = 'Loss'
        
    return row
    
matches_scores_final = matches_scores_temp.apply(lambda x: result(x), axis=1).drop('winner', axis=1)
matches_scores_final

In [ ]:
#matches_scores_final.to_csv("Match_Scores.csv")

## Pass accuracy

In [91]:
pass_acc_temp = events_tagged_last5.explode('split_tags').rename(columns={'split_tags':'Tag'})
pass_acc_temp = pass_acc_temp[pass_acc_temp['eventId']==8][['matchId', 'teamId', 'playerId', 'subEventId', 
                                                            'subEventName', 'eventSec', 'Tag']]

#getting the tag labels
pass_acc_temp = pass_acc_temp.merge(event_tag2name[['Tag','Label']], how="left").drop('Tag', axis=1)
pass_acc_temp

,matchId,teamId,playerId,subEventId,subEventName,eventSec,Label
0,2500039,1659,7989,85,Simple pass,1,accurate
1,2500039,1659,9279,85,Simple pass,1,accurate
2,2500039,1659,9277,85,Simple pass,1,accurate
3,2500039,1659,134102,85,Simple pass,1,accurate
4,2500039,1659,9285,85,Simple pass,1,accurate
...,...,...,...,...,...,...,...
49851,2500088,1624,14911,85,Simple pass,95,accurate
49852,2500088,1624,14911,85,Simple pass,95,accurate
49853,2500088,1624,54,80,Cross,96,Left
49854,2500088,1624,54,80,Cross,96,blocked


In [92]:
#categorizing time
def time_cat(x):
    if x >= 0 and x <= 15:
        return '0-15'
    elif x > 15 and x <= 30:
        return '16-30'
    elif x > 30 and x <= 45:
        return '31-45'
    elif x > 45 and x <= 60:
        return '46-60'
    elif x > 60 and x <= 75:
        return '61-75'
    else:
        return '76-90+'
pass_acc_temp['timeCat'] = pass_acc_temp['eventSec'].apply(lambda x: time_cat(x))
pass_acc_temp.head()

,matchId,teamId,playerId,subEventId,subEventName,eventSec,Label,timeCat
0,2500039,1659,7989,85,Simple pass,1,accurate,0-15
1,2500039,1659,9279,85,Simple pass,1,accurate,0-15
2,2500039,1659,9277,85,Simple pass,1,accurate,0-15
3,2500039,1659,134102,85,Simple pass,1,accurate,0-15
4,2500039,1659,9285,85,Simple pass,1,accurate,0-15


In [93]:
#finding accurate and not accurate passes
pass_acc_temp1 = pass_acc_temp.groupby(['matchId','teamId','playerId','subEventName','timeCat', 'Label']).agg({'subEventId':'count'}).reset_index().rename(columns={'subEventId':'count'})

pass_acc_temp1 = pass_acc_temp1[pass_acc_temp1['Label'].isin(['accurate', 'not accurate'])]
pass_acc_temp1.head()

,matchId,teamId,playerId,subEventName,timeCat,Label,count
2,2500039,1628,454,Cross,16-30,not accurate,1
4,2500039,1628,454,Cross,46-60,accurate,1
5,2500039,1628,454,Head pass,0-15,accurate,1
6,2500039,1628,454,Head pass,61-75,accurate,2
7,2500039,1628,454,Head pass,76-90+,accurate,1


In [94]:
#fiding pass accuracy
pass_accuracy = pass_acc_temp1.pivot(index=['matchId', 'teamId', 'playerId', 'subEventName', 'timeCat'], columns='Label', values='count').reset_index()
pass_accuracy = pass_accuracy.fillna(0)
pass_accuracy['accurate'] = pass_accuracy['accurate'].astype('int')
pass_accuracy['not accurate'] = pass_accuracy['not accurate'].astype(int)
pass_accuracy['passAccuracy'] = ((pass_accuracy['accurate'] / (pass_accuracy['accurate'] + pass_accuracy['not accurate']))*100).round(decimals=2)
pass_accuracy = pass_accuracy.drop(['accurate'], axis=1)
pass_accuracy

Label,matchId,teamId,playerId,subEventName,timeCat,not accurate,passAccuracy
0,2500039,1628,454,Cross,16-30,1,0.0
1,2500039,1628,454,Cross,46-60,0,100.0
2,2500039,1628,454,Head pass,0-15,0,100.0
3,2500039,1628,454,Head pass,61-75,0,100.0
4,2500039,1628,454,Head pass,76-90+,0,100.0
...,...,...,...,...,...,...,...
13655,2500088,1627,119951,Launch,76-90+,1,0.0
13656,2500088,1627,119951,Simple pass,0-15,0,100.0
13657,2500088,1627,119951,Simple pass,16-30,0,100.0
13658,2500088,1627,119951,Simple pass,46-60,0,100.0


In [95]:
#adding gameweek
pass_accuracy = pass_accuracy.merge(matches[['wyId', 'gameweek']].rename(columns={'wyId':'matchId'}), how='left')[['matchId', 'gameweek', 'teamId', 'playerId', 
                                                                                                   'subEventName', 'timeCat', 'not accurate', 'passAccuracy']]

pass_accuracy

,matchId,gameweek,teamId,playerId,subEventName,timeCat,not accurate,passAccuracy
0,2500039,33,1628,454,Cross,16-30,1,0.0
1,2500039,33,1628,454,Cross,46-60,0,100.0
2,2500039,33,1628,454,Head pass,0-15,0,100.0
3,2500039,33,1628,454,Head pass,61-75,0,100.0
4,2500039,33,1628,454,Head pass,76-90+,0,100.0
...,...,...,...,...,...,...,...,...
13655,2500088,37,1627,119951,Launch,76-90+,1,0.0
13656,2500088,37,1627,119951,Simple pass,0-15,0,100.0
13657,2500088,37,1627,119951,Simple pass,16-30,0,100.0
13658,2500088,37,1627,119951,Simple pass,46-60,0,100.0


In [96]:
#pass_accuracy.to_csv('pass_accuracy.csv')

In [97]:
player_position = pd.read_csv("player_positions.csv")[['wyId', 'shortName']].rename(columns={'wyId':'playerId', 'shortName':'shortName_sender'})

In [98]:
pass_accuracy = pass_accuracy.merge(player_position, how='left')[['matchId', 'gameweek', 'teamId', 'playerId', 'shortName_sender', 'subEventName', 'timeCat',
       'not accurate', 'passAccuracy']].rename(columns={'playerId':'playerId_sender'})

pass_accuracy

,matchId,gameweek,teamId,playerId_sender,shortName_sender,subEventName,timeCat,not accurate,passAccuracy
0,2500039,33,1628,454,P. van Aanholt,Cross,16-30,1,0.0
1,2500039,33,1628,454,P. van Aanholt,Cross,46-60,0,100.0
2,2500039,33,1628,454,P. van Aanholt,Head pass,0-15,0,100.0
3,2500039,33,1628,454,P. van Aanholt,Head pass,61-75,0,100.0
4,2500039,33,1628,454,P. van Aanholt,Head pass,76-90+,0,100.0
...,...,...,...,...,...,...,...,...,...
13655,2500088,37,1627,119951,Ahmed Hegazy,Launch,76-90+,1,0.0
13656,2500088,37,1627,119951,Ahmed Hegazy,Simple pass,0-15,0,100.0
13657,2500088,37,1627,119951,Ahmed Hegazy,Simple pass,16-30,0,100.0
13658,2500088,37,1627,119951,Ahmed Hegazy,Simple pass,46-60,0,100.0


In [99]:
pass_network = pd.read_csv("pass_network.csv")

accurate = pass_network[['matchId', 'teamId', 'playerId_sender', 'shortName_sender', 'subEventName', 'timeCat', 'Label']].groupby(['matchId', 'teamId', 'playerId_sender', 
                                                                                                                         'shortName_sender', 'subEventName', 'timeCat']).agg({'Label':'count'}).reset_index().rename(columns={'Label':'accurate'})

accurate

,matchId,teamId,playerId_sender,shortName_sender,subEventName,timeCat,accurate
0,2500039,1628,454,P. van Aanholt,Cross,46-60,2
1,2500039,1628,454,P. van Aanholt,Head pass,0-15,1
2,2500039,1628,454,P. van Aanholt,Head pass,61-75,2
3,2500039,1628,454,P. van Aanholt,Head pass,76-90+,2
4,2500039,1628,454,P. van Aanholt,High pass,61-75,1
...,...,...,...,...,...,...,...
9107,2500088,1627,119951,Ahmed Hegazy,Launch,0-15,1
9108,2500088,1627,119951,Ahmed Hegazy,Launch,46-60,2
9109,2500088,1627,119951,Ahmed Hegazy,Simple pass,0-15,3
9110,2500088,1627,119951,Ahmed Hegazy,Simple pass,16-30,2


In [100]:
pass_accuracy = pass_accuracy.merge(accurate, how='left').fillna(0)
pass_accuracy['passAccuracy'] = ((pass_accuracy['accurate'] / (pass_accuracy['accurate'] + pass_accuracy['not accurate']))*100).round(decimals=2)
pass_accuracy = pass_accuracy.dropna()
pass_accuracy = pass_accuracy.drop(['accurate'], axis=1)
pass_accuracy

,matchId,gameweek,teamId,playerId_sender,shortName_sender,subEventName,timeCat,not accurate,passAccuracy
0,2500039,33,1628,454,P. van Aanholt,Cross,16-30,1,0.0
1,2500039,33,1628,454,P. van Aanholt,Cross,46-60,0,100.0
2,2500039,33,1628,454,P. van Aanholt,Head pass,0-15,0,100.0
3,2500039,33,1628,454,P. van Aanholt,Head pass,61-75,0,100.0
4,2500039,33,1628,454,P. van Aanholt,Head pass,76-90+,0,100.0
...,...,...,...,...,...,...,...,...,...
13655,2500088,37,1627,119951,Ahmed Hegazy,Launch,76-90+,1,0.0
13656,2500088,37,1627,119951,Ahmed Hegazy,Simple pass,0-15,0,100.0
13657,2500088,37,1627,119951,Ahmed Hegazy,Simple pass,16-30,0,100.0
13658,2500088,37,1627,119951,Ahmed Hegazy,Simple pass,46-60,0,100.0


In [102]:
#pass_accuracy.to_csv('pass_accuracy_latest.csv')

## Second Tab
## Shots

In [ ]:
#events_tagged_last5[['eventName', 'subEventName']].drop_duplicates()

In [ ]:
events_tagged_last5[events_tagged_last5['eventName']=='Free Kick']['subEventName'].unique()

In [ ]:
#events_tagged_last5[(events_tagged_last5['eventName']=='Free Kick') & (events_tagged_last5['subEventName']=='Goal kick')]

In [ ]:
events_tagged_last5[(events_tagged_last5['matchId']==2500039) & (events_tagged_last5['subEventName']=='Free kick cross')]